In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


In [2]:
# %% [markdown]
# # 2. データ読み込みと前処理（GPU高速化）

# %%
# データパスの設定
file_path = 'output/06_【POS情報】店別－商品別実績_TX秋葉原駅_TX六町駅_TXつくば駅_20250709_20250722.csv'

# データ読み込み（cudf.pandasにより自動的にGPU加速）
print("📊 データ読み込み中...")
try:
    # UTF-8で試行
    df = pd.read_csv(file_path, encoding='utf-8')
    print(f"✅ データ読み込み成功（UTF-8）: {len(df):,}行")
except:
    try:
        # CP932で試行
        df = pd.read_csv(file_path, encoding='cp932')
        print(f"✅ データ読み込み成功（CP932）: {len(df):,}行")
    except Exception as e:
        print(f"❌ データ読み込みエラー: {e}")
        raise

# GPU使用時のメモリ情報表示
if GPU_AVAILABLE:
    try:
        import pynvml
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        print(f"📊 GPU メモリ使用状況: {info.used/1e9:.1f}/{info.total/1e9:.1f} GB")
    except:
        pass

# %%
# 基本的なデータクリーニング
# 日付型に変換
df['日付'] = pd.to_datetime(df['日付'])

# 店舗名の正規化
df['店舗'] = df['店舗'].str.replace('ファミリーマート　', '').str.replace('店', '')

# 不要データの除外
df = df[df['商品名'].notna() & (df['商品名'] != '不明') & (df['商品名'].str.strip() != '')]
df = df[~df['店舗'].str.contains('合計|小計', na=False)]

print(f"✅ クリーニング後: {len(df):,}行")
print(f"   店舗数: {df['店舗'].nunique()}")
print(f"   商品数: {df['商品名'].nunique()}")
print(f"   期間: {df['日付'].min()} ～ {df['日付'].max()}")

# %%
# 派生変数の作成（GPU加速により高速化）
print("📊 派生変数作成中...")

# 単価の計算
df['単価'] = np.where(df['売上数量'] > 0, df['売上金額'] / df['売上数量'], 0)
df['単価'] = df['単価'].replace([np.inf, -np.inf], np.nan).fillna(0)

# 販売率の計算
df['販売率'] = np.where(df['納品数量'] > 0, df['売上数量'] / df['納品数量'], 0)
df['販売率'] = df['販売率'].replace([np.inf, -np.inf], np.nan).fillna(0)

# 在庫回転日数の計算
df['在庫回転日数'] = np.where(
    df['売上数量'] > 0,
    df['納品数量'] / df['売上数量'],
    30  # 売上がない場合は30日とする
)
df['在庫回転日数'] = df['在庫回転日数'].clip(upper=30)

# 廃棄リスクスコア（販売率が低く、在庫回転日数が長い）
df['廃棄リスクスコア'] = (1 - df['販売率']) * df['在庫回転日数'] / 30

print("✅ 派生変数作成完了")
print(f"   平均単価: {df['単価'].mean():.0f}円")
print(f"   平均販売率: {df['販売率'].mean():.1%}")

📊 データ読み込み中...
✅ データ読み込み成功（UTF-8）: 13,344行
📊 GPU メモリ使用状況: 4.0/12.9 GB
✅ クリーニング後: 13,318行
   店舗数: 3
   商品数: 1112
   期間: 2025-07-09 00:00:00 ～ 2025-07-22 00:00:00
📊 派生変数作成中...


TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
# %% [markdown]
# # 3. 基本統計分析（GPU加速）

# %%
# 店舗別基本統計（groupby操作がGPU加速される）
print("📊 店舗別統計を計算中...")

store_stats = df.groupby('店舗').agg({
    '売上金額': ['sum', 'mean', 'std'],
    '売上数量': ['sum', 'mean'],
    '納品数量': ['sum', 'mean'],
    '商品名': 'nunique',
    '日付': 'nunique',
    '販売率': 'mean'
}).round(2)

# カラム名を整理
store_stats.columns = ['_'.join(col).strip() for col in store_stats.columns.values]
store_stats = store_stats.reset_index()

# 日販の計算
store_stats['日販'] = store_stats['売上金額_sum'] / store_stats['日付_nunique']

print("📊 店舗別基本統計:")
print(store_stats[['店舗', '日販', '売上金額_sum', '販売率_mean', '商品名_nunique']])

# %%
# 商品別基本統計（上位商品）
print("\n📊 商品別統計を計算中...")

product_stats = df.groupby('商品名').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '納品数量': 'sum',
    '販売率': 'mean',
    '在庫回転日数': 'mean',
    'フェイスくくり大分類': 'first',
    'PB/NBフラグ': 'first'
}).round(2)

# 売上順にソート
product_stats = product_stats.sort_values('売上金額', ascending=False)

print("\n📊 売上上位10商品:")
print(product_stats.head(10)[['売上金額', '販売率', '在庫回転日数']])

In [ ]:
# %% [markdown]
# # 4. 売上パターン分析（GPU高速化）

# %%
# 曜日別売上分析（大規模集計もGPU加速で高速）
print("📊 曜日別パターン分析中...")

weekday_analysis = df.groupby(['店舗', '曜日']).agg({
    '売上金額': ['sum', 'mean', 'count']
}).round(0)

# 曜日順序を定義
weekday_order = ['月', '火', '水', '木', '金', '土', '日']

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 店舗別曜日パターン
for store in df['店舗'].unique():
    store_weekday = weekday_analysis.loc[store]['売上金額']['mean']
    store_weekday = store_weekday.reindex(weekday_order)
    axes[0].plot(weekday_order, store_weekday, marker='o', label=store, linewidth=2)

axes[0].set_title('店舗別曜日売上パターン', fontsize=14, fontproperties=JP_FP)
axes[0].set_xlabel('曜日', fontproperties=JP_FP)
axes[0].set_ylabel('平均売上金額', fontproperties=JP_FP)
axes[0].legend(prop=JP_FP)
axes[0].grid(True, alpha=0.3)

# 全体の曜日パターン
overall_weekday = df.groupby('曜日')['売上金額'].mean().reindex(weekday_order)
bars = axes[1].bar(weekday_order, overall_weekday, color=COLORS[0])

# ピークと谷を特定
peak_day = overall_weekday.idxmax()
low_day = overall_weekday.idxmin()

# 色を変更
for i, day in enumerate(weekday_order):
    if day == peak_day:
        bars[i].set_color('#2ECC71')
    elif day == low_day:
        bars[i].set_color('#E74C3C')

axes[1].set_title('全体曜日別売上（ピーク:緑、谷:赤）', fontsize=14, fontproperties=JP_FP)
axes[1].set_xlabel('曜日', fontproperties=JP_FP)
axes[1].set_ylabel('平均売上金額', fontproperties=JP_FP)
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/10000):.0f}万'))

plt.tight_layout()
plt.show()

# 分析結果を辞書に保存
weekday_insights = {
    'peak_day': peak_day,
    'low_day': low_day,
    'peak_to_low_ratio': float(overall_weekday.max() / overall_weekday.min()),
    'pattern': overall_weekday.to_dict()
}

print(f"\n📊 曜日分析結果:")
print(f"   ピーク曜日: {peak_day}")
print(f"   最低曜日: {low_day}")
print(f"   ピーク/最低比: {weekday_insights['peak_to_low_ratio']:.2f}倍")

# %%
# 時間的トレンド分析
daily_trend = df.groupby('日付')['売上金額'].sum()

# トレンド分析
x = np.arange(len(daily_trend))
y = daily_trend.values
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# 可視化
plt.figure(figsize=(12, 5))
plt.plot(daily_trend.index, daily_trend.values, alpha=0.7, label='日次売上')

# トレンドライン
trend_line = slope * x + intercept
plt.plot(daily_trend.index, trend_line, 'r--', linewidth=2, 
         label=f'トレンド（傾き: {slope:.0f}円/日）')

# 移動平均
ma7 = daily_trend.rolling(window=7, center=True).mean()
plt.plot(daily_trend.index, ma7, 'g-', linewidth=2, label='7日移動平均')

plt.title('売上の時系列トレンド', fontsize=14, fontproperties=JP_FP)
plt.xlabel('日付', fontproperties=JP_FP)
plt.ylabel('売上金額', fontproperties=JP_FP)
plt.legend(prop=JP_FP)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

trend_insights = {
    'direction': 'increasing' if slope > 0 else 'decreasing',
    'daily_change': float(slope),
    'statistical_significance': p_value < 0.05,
    'r_squared': float(r_value ** 2)
}

print(f"\n📊 トレンド分析結果:")
print(f"   方向: {trend_insights['direction']}")
print(f"   日次変化: {trend_insights['daily_change']:.0f}円/日")
print(f"   統計的有意性: {trend_insights['statistical_significance']}")

In [ ]:
# %% [markdown]
# # 5. 在庫効率分析（GPU対応）

# %%
# 商品効率性マトリックス作成（大規模集計もGPU加速）
print("📊 在庫効率分析中...")

efficiency_data = df.groupby('商品名').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '納品数量': 'sum',
    '販売率': 'mean',
    '在庫回転日数': 'mean',
    '廃棄リスクスコア': 'mean',
    'フェイスくくり大分類': 'first',
    'PB/NBフラグ': 'first'
}).reset_index()

# 効率スコアの計算（売上貢献度 × 回転率）
efficiency_data['売上貢献度'] = efficiency_data['売上金額'] / efficiency_data['売上金額'].sum()
efficiency_data['効率スコア'] = (
    efficiency_data['売上貢献度'] * 1000 * 
    (1 / (efficiency_data['在庫回転日数'] + 1))
)

# %%
# 商品分類（4象限分析）
# 販売率と売上金額で分類
median_rate = efficiency_data['販売率'].median()
median_sales = efficiency_data['売上金額'].median()

efficiency_data['商品分類'] = ''
efficiency_data.loc[
    (efficiency_data['販売率'] >= median_rate) & 
    (efficiency_data['売上金額'] >= median_sales), '商品分類'
] = 'スター商品'

efficiency_data.loc[
    (efficiency_data['販売率'] < median_rate) & 
    (efficiency_data['売上金額'] >= median_sales), '商品分類'
] = '要改善商品'

efficiency_data.loc[
    (efficiency_data['販売率'] >= median_rate) & 
    (efficiency_data['売上金額'] < median_sales), '商品分類'
] = 'ニッチ商品'

efficiency_data.loc[
    (efficiency_data['販売率'] < median_rate) & 
    (efficiency_data['売上金額'] < median_sales), '商品分類'
] = '削減候補'

# 可視化
plt.figure(figsize=(10, 8))
colors_map = {
    'スター商品': '#2ECC71',
    '要改善商品': '#F39C12',
    'ニッチ商品': '#3498DB',
    '削減候補': '#E74C3C'
}

for category, color in colors_map.items():
    data = efficiency_data[efficiency_data['商品分類'] == category]
    plt.scatter(data['販売率'] * 100, data['売上金額'] / 10000,
                s=50, alpha=0.6, c=color, label=f'{category}（{len(data)}品）')

plt.axvline(x=median_rate * 100, color='gray', linestyle='--', alpha=0.5)
plt.axhline(y=median_sales / 10000, color='gray', linestyle='--', alpha=0.5)

plt.xlabel('販売率（%）', fontproperties=JP_FP)
plt.ylabel('売上金額（万円）', fontproperties=JP_FP)
plt.title('商品効率性マトリックス', fontsize=14, fontproperties=JP_FP)
plt.legend(prop=JP_FP)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 分類別統計
classification_stats = efficiency_data.groupby('商品分類').agg({
    '商品名': 'count',
    '売上金額': 'sum',
    '販売率': 'mean',
    '在庫回転日数': 'mean'
}).round(2)

print("📊 商品分類別統計:")
print(classification_stats)

# %%
# 削減候補商品の詳細分析
reduction_candidates = efficiency_data[
    efficiency_data['商品分類'] == '削減候補'
].sort_values('効率スコア')

# 削減による影響分析
total_reduction_sales = reduction_candidates['売上金額'].sum()
total_reduction_items = len(reduction_candidates)

print(f"\n📊 削減候補分析:")
print(f"   対象商品数: {total_reduction_items}品")
print(f"   影響売上額: {total_reduction_sales/10000:.0f}万円")
print(f"   全体に占める割合: {total_reduction_sales/efficiency_data['売上金額'].sum()*100:.1f}%")

# カテゴリ別削減候補
reduction_by_category = reduction_candidates.groupby('フェイスくくり大分類').agg({
    '商品名': 'count',
    '売上金額': 'sum'
}).sort_values('商品名', ascending=False)

print("\n📊 カテゴリ別削減候補:")
print(reduction_by_category.head(10))

In [ ]:
# %% [markdown]
# # 6. 気象影響分析（GPU加速）

# %%
# 天候別売上分析
print("📊 気象影響分析中...")

weather_analysis = df.groupby('天気').agg({
    '売上金額': ['sum', 'mean', 'count'],
    '売上数量': 'mean',
    '商品名': 'nunique'
}).round(0)

weather_analysis.columns = ['_'.join(col).strip() for col in weather_analysis.columns.values]

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 天候別平均売上
weather_avg = weather_analysis['売上金額_mean'].sort_values(ascending=False)
bars1 = axes[0].bar(weather_avg.index, weather_avg.values, color=COLORS[:len(weather_avg)])
axes[0].set_title('天候別平均売上', fontsize=14, fontproperties=JP_FP)
axes[0].set_ylabel('平均売上金額', fontproperties=JP_FP)
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/10000):.0f}万'))

# 数値表示
for i, (weather, value) in enumerate(weather_avg.items()):
    axes[0].text(i, value + value * 0.02, f'{int(value/10000, fontproperties=JP_FP)}万', ha='center')

# 気温と売上の相関
temp_corr_data = df.groupby('日付').agg({
    '売上金額': 'sum',
    '最高気温': 'first',
    '最低気温': 'first'
}).reset_index()

temp_corr_data['平均気温'] = (temp_corr_data['最高気温'] + temp_corr_data['最低気温']) / 2

# 散布図
scatter = axes[1].scatter(temp_corr_data['平均気温'], 
                         temp_corr_data['売上金額'] / 10000,
                         c=temp_corr_data.index, cmap='coolwarm', s=50, alpha=0.7)

# 相関係数
corr_coef = temp_corr_data['平均気温'].corr(temp_corr_data['売上金額'])

axes[1].set_title(f'気温と売上の関係（相関: {corr_coef:.3f}）', fontsize=14, fontproperties=JP_FP)
axes[1].set_xlabel('平均気温（℃）', fontproperties=JP_FP)
axes[1].set_ylabel('売上金額（万円）', fontproperties=JP_FP)

# トレンドライン
z = np.polyfit(temp_corr_data['平均気温'], temp_corr_data['売上金額'], 1)
p = np.poly1d(z)
axes[1].plot(temp_corr_data['平均気温'].sort_values(), 
             p(temp_corr_data['平均気温'].sort_values()) / 10000,
             "r--", linewidth=2, alpha=0.8)

plt.tight_layout()
plt.show()

# %%
# 気温感応商品の特定（並列処理でGPU高速化）
print("📊 気温感応商品の特定中...")

temp_sensitive_products = {}

# 売上上位100商品について気温相関を計算
top_products = product_stats.head(100).index

for product in top_products:
    product_data = df[df['商品名'] == product].groupby('日付').agg({
        '売上金額': 'sum',
        '最高気温': 'first'
    })
    
    if len(product_data) >= 5:
        corr = product_data['売上金額'].corr(product_data['最高気温'])
        if abs(corr) > 0.5:  # 相関が強い商品のみ
            temp_sensitive_products[product] = float(corr)

# ソートして表示
temp_sensitive_sorted = sorted(temp_sensitive_products.items(), 
                              key=lambda x: abs(x[1]), reverse=True)

print("📊 気温感応商品TOP10:")
for i, (product, corr) in enumerate(temp_sensitive_sorted[:10]):
    direction = "高温で売れる" if corr > 0 else "低温で売れる"
    print(f"{i+1}. {product}: {corr:.3f} ({direction})")

In [ ]:
# %% [markdown]
# # 7. カテゴリ・PB/NB分析（GPU高速化）

# %%
# カテゴリ別パフォーマンス（大規模集計もGPU加速）
print("📊 カテゴリ・PB/NB分析中...")

category_performance = df.groupby('フェイスくくり大分類').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '販売率': 'mean',
    '商品名': 'nunique',
    '在庫回転日数': 'mean'
}).sort_values('売上金額', ascending=False)

# PB/NB別パフォーマンス
pb_nb_performance = df.groupby('PB/NBフラグ').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '販売率': 'mean',
    '単価': 'mean',
    '商品名': 'nunique'
})

# 可視化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# カテゴリ別売上構成
top_categories = category_performance.head(10)
axes[0, 0].pie(top_categories['売上金額'], labels=top_categories.index, 
               autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('カテゴリ別売上構成（上位10）', fontsize=14, fontproperties=JP_FP)

# カテゴリ別効率性
axes[0, 1].scatter(top_categories['販売率'] * 100, 
                   top_categories['在庫回転日数'],
                   s=top_categories['売上金額'] / 10000,
                   alpha=0.6, c=range(len(top_categories)), cmap='viridis')

for i, (idx, row) in enumerate(top_categories.iterrows()):
    axes[0, 1].annotate(idx[:10], 
                        (row['販売率'] * 100, row['在庫回転日数'], fontproperties=JP_FP),
                        fontsize=8, alpha=0.7)

axes[0, 1].set_xlabel('販売率（%）', fontproperties=JP_FP)
axes[0, 1].set_ylabel('在庫回転日数', fontproperties=JP_FP)
axes[0, 1].set_title('カテゴリ別効率性（バブルサイズ=売上）', fontsize=14, fontproperties=JP_FP)
axes[0, 1].grid(True, alpha=0.3)

# PB/NB比較
pb_nb_data = pb_nb_performance.reset_index()
x = np.arange(len(pb_nb_data))
width = 0.35

bars1 = axes[1, 0].bar(x - width/2, pb_nb_data['売上金額'] / 10000, 
                       width, label='売上（万円）', color=COLORS[0])
bars2 = axes[1, 0].bar(x + width/2, pb_nb_data['販売率'] * 100, 
                       width, label='販売率（%）', color=COLORS[1])

axes[1, 0].set_xlabel('商品タイプ', fontproperties=JP_FP)
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(pb_nb_data['PB/NBフラグ'])
axes[1, 0].set_title('PB/NB比較', fontsize=14, fontproperties=JP_FP)
axes[1, 0].legend(prop=JP_FP)

# PB/NB単価比較
axes[1, 1].bar(pb_nb_data['PB/NBフラグ'], pb_nb_data['単価'], color=COLORS[2])
axes[1, 1].set_ylabel('平均単価（円）', fontproperties=JP_FP)
axes[1, 1].set_title('PB/NB平均単価比較', fontsize=14, fontproperties=JP_FP)

for i, (idx, row) in enumerate(pb_nb_data.iterrows()):
    axes[1, 1].text(i, row['単価'] + 5, f"{row['単価']:.0f}円", ha='center', fontproperties=JP_FP)

plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# # 8. 機会損失分析（GPU並列処理）

# %%
# 品薄による機会損失
print("📊 機会損失分析中...")

shortage_products = efficiency_data[efficiency_data['販売率'] >= 0.9].copy()
shortage_products['推定損失額'] = shortage_products['売上金額'] * 0.1  # 10%の追加売上可能と仮定

total_shortage_loss = shortage_products['推定損失額'].sum()

# 過剰在庫コスト
excess_products = efficiency_data[efficiency_data['販売率'] < 0.3].copy()
excess_products['過剰在庫数'] = excess_products['納品数量'] - excess_products['売上数量']
excess_products['推定コスト'] = excess_products['過剰在庫数'] * excess_products['売上金額'] / excess_products['売上数量'] * 0.3

total_excess_cost = excess_products['推定コスト'].sum()

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 機会損失サマリー
loss_data = {
    '品薄による機会損失': total_shortage_loss / 10000,
    '過剰在庫コスト': total_excess_cost / 10000
}

bars = axes[0].bar(loss_data.keys(), loss_data.values(), color=['#E74C3C', '#F39C12'])
axes[0].set_ylabel('金額（万円）', fontproperties=JP_FP)
axes[0].set_title('改善可能な損失額', fontsize=14, fontproperties=JP_FP)

for i, (key, value) in enumerate(loss_data.items()):
    axes[0].text(i, value + value * 0.05, f'{value:.0f}万円', ha='center', fontproperties=JP_FP)

total_opportunity = sum(loss_data.values())
axes[0].text(0.5, 0.95, f'合計改善機会: {total_opportunity:.0f}万円', 
             transform=axes[0].transAxes, ha='center', fontsize=12,
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5, fontproperties=JP_FP))

# 商品数の内訳
opportunity_breakdown = {
    '品薄商品': len(shortage_products),
    '過剰在庫商品': len(excess_products),
    '適正商品': len(efficiency_data) - len(shortage_products) - len(excess_products)
}

axes[1].pie(opportunity_breakdown.values(), labels=opportunity_breakdown.keys(),
            autopct='%1.0f%%', startangle=90, colors=['#E74C3C', '#F39C12', '#2ECC71'])
axes[1].set_title('商品在庫状況の内訳', fontsize=14, fontproperties=JP_FP)

plt.tight_layout()
plt.show()

print(f"📊 機会損失分析結果:")
print(f"   品薄による損失: {total_shortage_loss/10000:.0f}万円（{len(shortage_products)}商品）")
print(f"   過剰在庫コスト: {total_excess_cost/10000:.0f}万円（{len(excess_products)}商品）")
print(f"   合計改善機会: {(total_shortage_loss + total_excess_cost)/10000:.0f}万円")

In [ ]:
# %% [markdown]
# # 9. 動的な発注最適化提案（GPU対応）

# %%
# 商品クラスタリング（発注戦略用）
print("📊 発注最適化クラスタリング中...")

# 特徴量の準備
cluster_features = efficiency_data[['販売率', '在庫回転日数', '売上貢献度', '効率スコア']].copy()

# 欠損値処理
cluster_features = cluster_features.fillna(cluster_features.median())

# GPU対応のクラスタリング（cuMLが利用可能な場合）
if GPU_AVAILABLE:
    try:
        from cuml.preprocessing import StandardScaler as cuStandardScaler
        from cuml.cluster import KMeans as cuKMeans
        
        # 標準化
        scaler = cuStandardScaler()
        features_scaled = scaler.fit_transform(cluster_features)
        
        # K-meansクラスタリング（GPU）
        n_clusters = 5
        kmeans = cuKMeans(n_clusters=n_clusters, random_state=42)
        efficiency_data['発注クラスター'] = kmeans.fit_predict(features_scaled)
        
        print("✅ GPU版K-meansクラスタリング完了")
    except:
        # CPU版にフォールバック
        from sklearn.preprocessing import StandardScaler
        from sklearn.cluster import KMeans
        
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(cluster_features)
        
        n_clusters = 5
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        efficiency_data['発注クラスター'] = kmeans.fit_predict(features_scaled)
        
        print("⚠️ CPU版K-meansクラスタリング完了")
else:
    # CPU版を使用
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(cluster_features)
    
    n_clusters = 5
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    efficiency_data['発注クラスター'] = kmeans.fit_predict(features_scaled)

# クラスター別特性分析
cluster_profiles = efficiency_data.groupby('発注クラスター').agg({
    '売上金額': ['mean', 'sum'],
    '販売率': 'mean',
    '在庫回転日数': 'mean',
    '商品名': 'count'
}).round(2)

# %%
# クラスターごとの発注戦略を動的に決定
ordering_strategies = {}

for cluster in range(n_clusters):
    cluster_data = cluster_profiles.loc[cluster]
    
    # 各指標を取得
    avg_sales = cluster_data[('売上金額', 'mean')]
    avg_rate = cluster_data[('販売率', 'mean')]
    avg_turnover = cluster_data[('在庫回転日数', 'mean')]
    item_count = cluster_data[('商品名', 'count')]
    
    # 動的に戦略を決定
    if avg_rate > 0.8 and avg_turnover < 3:
        strategy = "発注量20-30%増加"
        reason = "高回転・高販売率"
        priority = "高"
    elif avg_rate > 0.6 and avg_sales > cluster_profiles[('売上金額', 'mean')].median():
        strategy = "現状維持・微調整"
        reason = "安定的なパフォーマンス"
        priority = "中"
    elif avg_rate < 0.3:
        strategy = "発注量50%削減または廃止"
        reason = "低販売率"
        priority = "高"
    elif avg_turnover > 7:
        strategy = "発注量30%削減"
        reason = "在庫滞留"
        priority = "高"
    else:
        strategy = "販促強化後に再評価"
        reason = "改善余地あり"
        priority = "中"
    
    ordering_strategies[f'クラスター{cluster}'] = {
        '商品数': int(item_count),
        '平均販売率': float(avg_rate),
        '平均回転日数': float(avg_turnover),
        '戦略': strategy,
        '理由': reason,
        '優先度': priority
    }

# 戦略の表示
print("📊 動的発注戦略（クラスター分析に基づく）:")
for cluster, strategy in ordering_strategies.items():
    print(f"\n{cluster}（{strategy['商品数']}商品）:")
    print(f"  販売率: {strategy['平均販売率']:.1%}")
    print(f"  回転日数: {strategy['平均回転日数']:.1f}日")
    print(f"  戦略: {strategy['戦略']}")
    print(f"  理由: {strategy['理由']}")

In [ ]:
# %% [markdown]
# # 10. 統合ダッシュボード（GPU最適化）

# %%
# 統合KPIダッシュボード作成
print("📊 統合ダッシュボード生成中...")

fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# 1. 店舗別KPI
ax1 = fig.add_subplot(gs[0, :2])
store_kpi_data = store_stats[['店舗', '日販', '販売率_mean']]
x = np.arange(len(store_kpi_data))
width = 0.35

ax1_twin = ax1.twinx()
bars1 = ax1.bar(x, store_kpi_data['日販'] / 10000, width, label='日販（万円）', color=COLORS[0])
line1 = ax1_twin.plot(x, store_kpi_data['販売率_mean'] * 100, 'o-', 
                      color=COLORS[1], linewidth=2, markersize=10, label='販売率（%）')

ax1.set_xlabel('店舗', fontproperties=JP_FP)
ax1.set_ylabel('日販（万円）', color=COLORS[0], fontproperties=JP_FP)
ax1_twin.set_ylabel('販売率（%）', color=COLORS[1], fontproperties=JP_FP)
ax1.set_title('店舗別KPI', fontsize=16, fontproperties=JP_FP)
ax1.set_xticks(x)
ax1.set_xticklabels(store_kpi_data['店舗'])

# 2. 機会損失分析
ax2 = fig.add_subplot(gs[0, 2:])
loss_values = list(loss_data.values())
loss_labels = [f'{k}\n{v:.0f}万円' for k, v in loss_data.items()]
bars2 = ax2.bar(range(len(loss_values)), loss_values, color=['#E74C3C', '#F39C12'])
ax2.set_ylabel('金額（万円）', fontproperties=JP_FP)
ax2.set_title(f'改善機会: 合計{sum(loss_values, fontproperties=JP_FP):.0f}万円', fontsize=16)
ax2.set_xticks(range(len(loss_values)))
ax2.set_xticklabels(loss_labels)

# 3. 商品効率マトリックス（簡略版）
ax3 = fig.add_subplot(gs[1, :2])
for category, color in colors_map.items():
    data = efficiency_data[efficiency_data['商品分類'] == category]
    if len(data) > 0:
        ax3.scatter(data['販売率'].head(50) * 100, 
                   data['売上金額'].head(50) / 10000,
                   s=30, alpha=0.6, c=color, label=category)

ax3.set_xlabel('販売率（%）', fontproperties=JP_FP)
ax3.set_ylabel('売上金額（万円）', fontproperties=JP_FP)
ax3.set_title('商品分類マトリックス', fontsize=16, fontproperties=JP_FP)
ax3.legend(loc='best', fontsize=8, prop=JP_FP)
ax3.grid(True, alpha=0.3)

# 4. 曜日パターン
ax4 = fig.add_subplot(gs[1, 2:])
weekly_values = [weekday_insights['pattern'][day] for day in weekday_order]
bars4 = ax4.bar(weekday_order, weekly_values, color=COLORS[2])

# ピークと谷を強調
for i, day in enumerate(weekday_order):
    if day == weekday_insights['peak_day']:
        bars4[i].set_color('#2ECC71')
    elif day == weekday_insights['low_day']:
        bars4[i].set_color('#E74C3C')

ax4.set_xlabel('曜日', fontproperties=JP_FP)
ax4.set_ylabel('平均売上', fontproperties=JP_FP)
ax4.set_title('曜日別売上パターン', fontsize=16, fontproperties=JP_FP)
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/10000):.0f}万'))

# 5. 動的推奨事項
ax5 = fig.add_subplot(gs[2, :])
ax5.axis('off')

# 推奨事項テキスト
recommendations_text = "【データ分析に基づく動的推奨事項】\n\n"

# 優先度の高い施策を抽出
high_priority_actions = []

# 1. 品薄対策
if total_shortage_loss > 100000:
    high_priority_actions.append({
        'action': f'{len(shortage_products)}商品の発注量を20-30%増加',
        'impact': f'{total_shortage_loss/10000:.0f}万円の機会損失削減',
        'priority': '高'
    })

# 2. 過剰在庫対策
if total_excess_cost > 50000:
    high_priority_actions.append({
        'action': f'{len(excess_products)}商品の発注量を段階的に削減',
        'impact': f'{total_excess_cost/10000:.0f}万円のコスト削減',
        'priority': '高'
    })

# 3. 曜日対策
if weekday_insights['peak_to_low_ratio'] > 1.5:
    high_priority_actions.append({
        'action': f"{weekday_insights['peak_day']}の発注を{(weekday_insights['peak_to_low_ratio']-1)*100:.0f}%増加",
        'impact': '売上機会の最大化',
        'priority': '中'
    })

# 4. トレンド対策
if trend_insights['direction'] == 'decreasing' and trend_insights['statistical_significance']:
    high_priority_actions.append({
        'action': '売上減少トレンドへの対策（品揃え・価格見直し）',
        'impact': f'日次{abs(trend_insights["daily_change"]):.0f}円の減少を食い止める',
        'priority': '高'
    })

# テキスト生成
for i, action in enumerate(high_priority_actions[:5], 1):
    recommendations_text += f"{i}. {action['action']}\n"
    recommendations_text += f"   期待効果: {action['impact']}\n"
    recommendations_text += f"   優先度: {action['priority']}\n\n"

ax5.text(0.05, 0.95, recommendations_text, transform=ax5.transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3, fontproperties=JP_FP))

plt.suptitle('動的ビジネスインサイト統合ダッシュボード（GPU加速）', fontsize=20, fontproperties=JP_FP)
plt.tight_layout()
plt.show()

# GPUメモリ使用状況の表示
if GPU_AVAILABLE:
    try:
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        print(f"\n📊 GPU メモリ使用状況（処理後）: {info.used/1e9:.1f}/{info.total/1e9:.1f} GB")
    except:
        pass

In [ ]:
# %% [markdown]
# # 11. LLM用構造化データ出力

# %%
# 全ての分析結果を統合
print("📊 分析結果を構造化データに変換中...")

integrated_insights = {
    "analysis_metadata": {
        "analysis_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "gpu_enabled": GPU_AVAILABLE,
        "data_period": {
            "start": df['日付'].min().strftime("%Y-%m-%d"),
            "end": df['日付'].max().strftime("%Y-%m-%d"),
            "days": int((df['日付'].max() - df['日付'].min()).days) + 1
        },
        "data_scope": {
            "total_records": int(len(df)),
            "stores": int(df['店舗'].nunique()),
            "products": int(df['商品名'].nunique()),
            "categories": int(df['フェイスくくり大分類'].nunique())
        }
    },
    
    "key_metrics": {
        "store_performance": store_stats[['店舗', '日販', '販売率_mean']].to_dict('records'),
        "total_daily_sales": float(store_stats['日販'].sum()),
        "average_sales_rate": float(df['販売率'].mean()),
        "opportunity_analysis": {
            "shortage_loss": float(total_shortage_loss),
            "excess_cost": float(total_excess_cost),
            "total_opportunity": float(total_shortage_loss + total_excess_cost),
            "shortage_products_count": int(len(shortage_products)),
            "excess_products_count": int(len(excess_products))
        }
    },
    
    "patterns_discovered": {
        "weekly_pattern": weekday_insights,
        "time_trend": trend_insights,
        "weather_impact": {
            "temperature_correlation": float(corr_coef),
            "best_weather": weather_avg.index[0],
            "weather_sales_impact": float(weather_avg.iloc[0] / weather_avg.iloc[-1])
        },
        "temperature_sensitive_products": temp_sensitive_products
    },
    
    "product_insights": {
        "classification_summary": classification_stats.to_dict(),
        "reduction_candidates": {
            "count": int(total_reduction_items),
            "impact_sales": float(total_reduction_sales),
            "percentage_of_total": float(total_reduction_sales/efficiency_data['売上金額'].sum()*100)
        }
    },
    
    "category_insights": {
        "top_categories": category_performance.head(5)['売上金額'].to_dict(),
        "pb_nb_comparison": pb_nb_performance.to_dict()
    },
    
    "ordering_strategies": ordering_strategies,
    
    "priority_actions": [
        {
            "action": action['action'],
            "expected_impact": action['impact'],
            "priority": action['priority']
        }
        for action in high_priority_actions
    ],
    
    "processing_info": {
        "gpu_used": GPU_AVAILABLE,
        "processing_time": "最適化済み（GPU加速）" if GPU_AVAILABLE else "標準処理"
    }
}

# JSON形式で保存（オプション）
output_path = Path('analysis_results_gpu.json')
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(integrated_insights, f, ensure_ascii=False, indent=2, default=str)

print("✅ 分析結果をJSON形式で保存しました")
print(f"   ファイル: {output_path}")

# %%
# 分析サマリーの表示
print("\n" + "="*80)
print("📊 分析結果サマリー（LLM解釈用）")
print("="*80)

print(f"\n1. 基本情報:")
print(f"   分析期間: {integrated_insights['analysis_metadata']['data_period']['days']}日間")
print(f"   総売上: {integrated_insights['key_metrics']['total_daily_sales']/10000:.0f}万円/日")
print(f"   平均販売率: {integrated_insights['key_metrics']['average_sales_rate']:.1%}")
print(f"   GPU使用: {'✅ 有効' if GPU_AVAILABLE else '❌ 無効'}")

print(f"\n2. 改善機会:")
print(f"   合計: {integrated_insights['key_metrics']['opportunity_analysis']['total_opportunity']/10000:.0f}万円")
print(f"   品薄商品: {integrated_insights['key_metrics']['opportunity_analysis']['shortage_products_count']}品")
print(f"   過剰在庫: {integrated_insights['key_metrics']['opportunity_analysis']['excess_products_count']}品")

print(f"\n3. 発見されたパターン:")
print(f"   曜日ピーク: {integrated_insights['patterns_discovered']['weekly_pattern']['peak_day']}")
print(f"   売上トレンド: {integrated_insights['patterns_discovered']['time_trend']['direction']}")
print(f"   気温相関: {integrated_insights['patterns_discovered']['weather_impact']['temperature_correlation']:.3f}")

print(f"\n4. 優先アクション数: {len(integrated_insights['priority_actions'])}")

print("\n※このデータをLLMに入力することで、さらに詳細な分析と提案が可能です")

# %%
# 分析完了メッセージ
print("\n" + "="*80)
print("✅ 動的探索的データ分析が完了しました！（GPU最適化版）")
print("="*80)
print("\n処理の特徴:")
print("- cudf.pandasによる透過的なGPU加速")
print("- 大規模データの高速集計・分析")
print("- 日本語対応の可視化")
print("- LLM向け構造化データ出力")
print("\n次のステップ:")
print("1. analysis_results_gpu.jsonをLLMに入力して詳細な解釈を得る")
print("2. 予測モデルの構築に進む")
print("3. 特定の商品や店舗の深堀り分析を実施")
print("\nGPU環境により、より大規模なデータでも高速な分析が可能です！")